In [11]:
api_key

In [4]:
from openai import OpenAI
from pprint import pprint
from dotenv import load_dotenv
import httpx, os

# Load the .env file
load_dotenv("3ds.env")

# Create a custom httpx client that uses your corporate certificate
custom_http_client = httpx.Client(verify="ExaleadRootCAG2-chain.pem")

# Load environment variables from .env file
api_key = os.getenv("DS_OPENAI_API_KEY")
base_url = os.getenv("FM_GATEWAY_BASE_URL")

client = OpenAI(
    api_key=api_key,
    base_url=base_url,
    http_client=custom_http_client
)

response = client.models.list()

pprint(response.to_dict())

{'data': [{'capabilities': {},
           'context_length': 31999,
           'created': 1760517960,
           'id': 'mistralai/Magistral-Small-2509',
           'input_types': ['text'],
           'object': 'model',
           'owned_by': 'proxem',
           'size': '24B',
           'usages': ['text_generation']},
          {'capabilities': {},
           'context_length': 63999,
           'created': 1760520399,
           'id': 'meta-llama/Llama-3.1-8B-Instruct',
           'input_types': ['text'],
           'object': 'model',
           'owned_by': 'proxem',
           'size': '8B',
           'usages': ['text_generation']},
          {'capabilities': {},
           'context_length': 512,
           'created': 1760520481,
           'id': 'BAAI/bge-reranker-large',
           'input_types': ['text'],
           'object': 'model',
           'owned_by': 'proxem',
           'size': '1b',
           'usages': ['reranking']},
          {'capabilities': {},
           'context_leng

In [19]:
supplier_name = "Milan Creative Collectibles S.r.l."  # Replace with actual supplier name if needed
supplier_id = "uuid:6901bbfc-bac6-452c-a1ac-a861655e3150"  # Replace with actual supplier ID if needed

usage = "Pilot" # Options: "Pilot", "Production", "Development"
usage_id = "dsmatdata:pilot_usageStatus" # Options: "dsmatdata:pilot_usageStatus", "dsmatdata:production_usageStatus", "dsmatdata:development_usageStatus"

usage_restriction = "dsmatdata:none_usageRestriction" # Options: "dsmatdata:blockForNew_usageRestriction", "dsmatdata:blockForAll_usageRestriction", "dsmatdata:warning_usageRestriction", "dsmatdata:none_usageRestriction"

In [20]:
system_prompt = """
You are an expert materials data extraction model.
Your job is to analyze a material datasheet (provided as a PDF) and extract *all* structured material data
into a predefined JSON schema. Be precise and consistent.

Follow this JSON structure exactly, keeping the order of fields the same:
{{
  "name": "str",
  "description": "str",
  "iupacName": "str",
  "casNumber": "str",
  "recipeNumber": "str",
  "synonym": ["str"],
  "hasAssignedIdentifier": [
    {{
      "identifier": "str",
      "hasIdentifierType": {{
        "id": "str"
      }}
    }}
  ],
  "hasConstituency": {{
    "id": "str",
    "hasMixtureRatio": [
      {{
        "ratioMaximum": "float",
        "ratioMinimum": "float",
        "ratiotarget": "float",
        "ratioMaximumInclusive": "bool",
        "ratioMinimumInclusive": "bool",
        "restOfMixture": "bool",
        "offLabel": "bool",
        "primaryConstituent": "bool",
        "hasActiveMaterialFunction": [
          {{
            "id": "str"
          }}
        ],
        "hasRatioSubstance": {{
          "id": "str"
        }}
      }}
    ]
  }},
  "hasGrade": [
    {{
      "id": "str"
    }}
  ],
  "hasMaterialFunction": [
    {{
      "id": "str"
    }}
  ],
  "hasMaterialType": {{
    "id": "str"
  }},
  "hasSource": {{
    "hasSupplier": {{
      "supplierId": "str",
      "supplierName": "str"
    }},
    "supplierPartNumber": "str",
    "tradeName": "str"
  }},
  "groups": [
    {{
      "id": "str"
    }}
  ],
  "hasAlternative": [
    {{
      "id": "str"
    }}
  ],
  "hasFormulationSet": {{
    "hasAppearance": {{
      "val": "str"
    }},
    "hasAqueousSolubility": {{
      "val": "float"
    }},
    "hasAutoIgnitionTemperature": {{
      "val": "float"
    }},
    "hasBoilingPoint": {{
      "val": "float"
    }},
    "hasColorDescription": {{
      "val": "str"
    }},
    "hasDensity": {{
      "val": "float"
    }},
    "hasDynamicViscosity": {{
      "val": "float"
    }},
    "hasFlashPoint": {{
      "val": "float"
    }},
    "hasMeltingPoint": {{
      "val": "float"
    }},
    "hasPhysicalForm": {{
      "val": {{
        "id": "str"
      }}
    }},
    "hasSpecificGravity": {{
      "val": "float"
    }},
    "hasStandardCost": {{
      "val": "float"
    }},
    "hasUsageRestriction": {{
      "val": {{
        "id": "{2}"
      }},
      "message": "str"
    }},
    "hasUsageStatus": {{
      "val": {{
        "id": "{3}"
      }}
    }},
    "hasVaporPressure": {{
      "val": "float"
    }}
  }},
  "hasSecureCollection": [
    {{
      "id": "dsmatdata:secure_collection_standard"
    }}
  ]
}}

Rules:
- Output must be *strictly valid JSON*.
- Do not include any text outside the JSON.
- Extract name, description, iupacName, casNumber, and synonyms directly from the datasheet.
- If iupacName is not available, provide it from your chemical knowledge.
- If no synonyms are extracted from the document, use your chemical knowledge and provide 3-5 synonyms.
- Keep recipeNumber empty.
- Only include assigned identifier type IDs from the following mapping:
      {{
        'REACH Registration Number': 'dsmatdata:REACH_Number_IDT',
        'Total Materia ID': 'dsmatdata:TotalMateria_IDT',
        'ATCC': 'dsmatdata:ATCC_IDT',
        'Compendial Name': 'dsmatdata:Compendial_Name_IDT',
        'Colour Index Constitution Number': 'dsmatdata:CICN_IDT',
        'CISPro Number': 'dsmatdata:CISPro_Number_IDT',
        'MDL Number': 'dsmatdata:MDL_Number_IDT',
        'FoodChain ID': 'dsmatdata:Food_Chain_IDT',
        'FDA UNII': 'dsmatdata:FDA_UNII_IDT',
        'SPOR Substance ID': 'dsmatdata:SPOR_Substance_IDT',
        'Enterprise Identifier': 'dsmatdata:Enterprise_Identifier_IDT',
        'CAS Number': 'dsmatdata:CAS_Number_IDT',
        'EC Number': 'dsmatdata:EC_Number_IDT',
        'INCI Name': 'dsmatdata:INCI_Name_IDT'
      }}

- Keep hasConstituency empty.
- Only include grade IDs from the following mapping, default value is "dsmatdata:laboratory_grade":
      {{
        "Laboratory": "dsmatdata:laboratory_grade",
        "Technical": "dsmatdata:technical_grade",
        "Analytical Research": "dsmatdata:analytical_research_grade",
        "JP": "dsmatdata:jp_grade",
        "NF": "dsmatdata:nf_grade",
        "Purified": "dsmatdata:purified_grade",
        "USP-NF": "dsmatdata:usp_nf_grade",
        "GRC": "dsmatdata:grc_grade",
        "USP": "dsmatdata:usp_grade",
        "EP (Ph. Eur.)": "dsmatdata:ep_grade",
        "FCC": "dsmatdata:fcc_grade",
        "Reagent": "dsmatdata:reagent_grade",
        "BP": "dsmatdata:bp_grade",
        "ACS": "dsmatdata:acs_grade"
      }}

- Only include material function IDs from the defined mapping:
      {{
        "Binder": "dsmatdata:binder_matfunc",
        "Surface Modifier": "dsmatdata:surfacemodifier_matfunc",
        "Processing Aid": "dsmatdata:processingaid_matfunc",
        "Viscosity Decreasing Agent": "dsmatdata:viscositydecreasingagent_matfunc",
        "Buffering Agent": "dsmatdata:bufferingagent_matfunc",
        "Skin Conditioning Agent - Miscellaneous": "dsmatdata:skinconditioningagentmiscellaneous_matfunc",
        "Film Former": "dsmatdata:filmformer_matfunc",
        "Fluorescent Brightening Agent": "dsmatdata:fluorescentbrighteningagent_matfunc",
        "Antimicrobial Agent": "dsmatdata:antimicrobialagent_matfunc",
        "Carrier Agent": "dsmatdata:carrieragent_matfunc",
        "Stain Remover": "dsmatdata:stainremover_matfunc",
        "Cosmetic Biocide": "dsmatdata:cosmeticbiocide_matfunc",
        "Surfactant": "dsmatdata:surfactant_matfunc",
        "Water Repellant": "dsmatdata:waterrepellant_matfunc",
        "Active Ingredient": "dsmatdata:activepharmaceuticalingredient_matfunc",
        "Toner": "dsmatdata:toner_matfunc",
        "Hueing Dye": "dsmatdata:hueingdye_matfunc",
        "Brightener": "dsmatdata:brightener_matfunc",
        "Preservative": "dsmatdata:preservative_matfunc",
        "Acidity Regulator": "dsmatdata:acidityregulator_matfunc",
        "Substrate": "dsmatdata:substrate_matfunc",
        "Cationic Promoter": "dsmatdata:cationicpromoter_matfunc",
        "Conditioning Agent": "dsmatdata:conditioningagent_matfunc",
        "Propellant": "dsmatdata:propellant_matfunc",
        "Adhesive": "dsmatdata:adhesive_matfunc",
        "Bleaching Agent": "dsmatdata:bleachingagent_matfunc",
        "Softening Agent": "dsmatdata:softeningagent_matfunc",
        "Enzyme": "dsmatdata:enzyme_matfunc",
        "Respiratory Agent": "dsmatdata:respiratoryagent_matfunc",
        "Sunscreen Agent": "dsmatdata:sunscreenagent_matfunc",
        "Sweetening Agent": "dsmatdata:sweeteningagent_matfunc",
        "Antidandruff Agent": "dsmatdata:antidandruffagent_matfunc",
        "Surfactant - Nonionic": "dsmatdata:surfactantnonionic_matfunc",
        "Dye Fixing Agent": "dsmatdata:dyefixingagent_matfunc",
        "Fixative": "dsmatdata:fixative_matfunc",
        "Tablet Coating Agent": "dsmatdata:tabletcoatingagent_matfunc",
        "Antiperspirant Agent": "dsmatdata:antiperspirantagent_matfunc",
        "Ion Exchange Agent": "dsmatdata:ionexchangeagent_matfunc",
        "Sequestering Agent": "dsmatdata:sequesteringagent_matfunc",
        "Pigment Dispersant": "dsmatdata:pigmentdispersant_matfunc",
        "Perfume": "dsmatdata:perfume_matfunc",
        "pH Adjuster": "dsmatdata:phadjuster_matfunc",
        "Viscosity Increasing Agent - Aqueous": "dsmatdata:viscosityincreasingagentaqueous_matfunc",
        "Artificial Nail Builder": "dsmatdata:artificialnailbuilder_matfunc",
        "External Analgesic": "dsmatdata:externalanalgesic_matfunc",
        "Biocatalyst": "dsmatdata:biocatalyst_matfunc",
        "Ultraviolet Light Absorber": "dsmatdata:ultravioletlightabsorber_matfunc",
        "Water Conditioning Agent": "dsmatdata:waterconditioningagent_matfunc",
        "Elastomer": "dsmatdata:elastomer_matfunc",
        "Oral Care Agent": "dsmatdata:oralcareagent_matfunc",
        "Anticaking Agent": "dsmatdata:anticakingagent_matfunc",
        "Bulking Agent": "dsmatdata:bulkingagent_matfunc",
        "Skin Conditioning Agent": "dsmatdata:skinconditioningagent_matfunc",
        "Colorant": "dsmatdata:colorant_matfunc",
        "Moisture Barrier": "dsmatdata:moisturebarrier_matfunc",
        "Hair Colorant": "dsmatdata:haircolorant_matfunc",
        "Stabilizer": "dsmatdata:stabilizer_matfunc",
        "Strength Additive": "dsmatdata:strengthadditive_matfunc",
        "Cooling Agent": "dsmatdata:coolingagent_matfunc",
        "Gelling Material": "dsmatdata:gellingmaterial_matfunc",
        "Antifoaming Agent": "dsmatdata:antifoamingagent_matfunc",
        "Hydrotrope": "dsmatdata:hydrotrope_matfunc",
        "Slip Modifier": "dsmatdata:slipmodifier_matfunc",
        "Hair Conditioning Agent": "dsmatdata:hairconditioningagent_matfunc",
        "Humectant": "dsmatdata:humectant_matfunc",
        "Fragrance Ingredient": "dsmatdata:fragranceingredient_matfunc",
        "Surfactant - Amphoteric": "dsmatdata:surfactantamphoteric_matfunc",
        "Additive": "dsmatdata:additive_matfunc",
        "Chelating Agent": "dsmatdata:chelatingagent_matfunc",
        "Fabric Softener": "dsmatdata:fabricsoftener_matfunc",
        "Protective Coating": "dsmatdata:protectivecoating_matfunc",
        "Skin Conditioning Agent - Occlusive": "dsmatdata:skinconditioningagentocclusive_matfunc",
        "Structurant": "dsmatdata:structurant_matfunc",
        "Wetting Agent": "dsmatdata:wettingagent_matfunc",
        "Diluent": "dsmatdata:diluent_matfunc",
        "Coagulant": "dsmatdata:coagulant_matfunc",
        "Epilating Agent": "dsmatdata:epilatingagent_matfunc",
        "Anti-Soil Redeposition Agent": "dsmatdata:antisoilredepositionagent_matfunc",
        "Curing Agent": "dsmatdata:curingagent_matfunc",
        "Bleaching Activator": "dsmatdata:bleachingactivator_matfunc",
        "Reducing Agent": "dsmatdata:reducingagent_matfunc",
        "Molding Material": "dsmatdata:moldingmaterial_matfunc",
        "Glitter": "dsmatdata:glitter_matfunc",
        "Surfactant - Foam Booster": "dsmatdata:surfactantfoambooster_matfunc",
        "Solvent": "dsmatdata:solvent_matfunc",
        "Denaturant": "dsmatdata:denaturant_matfunc",
        "Pesticide": "dsmatdata:pesticide_matfunc",
        "Surfactant - Cationic": "dsmatdata:surfactantcationic_matfunc",
        "Antiacne Agent": "dsmatdata:antiacneagent_matfunc",
        "Hydrating Agent": "dsmatdata:hydratingagent_matfunc",
        "Leavening Agent": "dsmatdata:leaveningagent_matfunc",
        "Disintegrant": "dsmatdata:disintegrant_matfunc",
        "Antibacterial Agent": "dsmatdata:antibacterialagent_matfunc",
        "Soap Base": "dsmatdata:soapbase_matfunc",
        "Skin Lubricant": "dsmatdata:skinlubricant_matfunc",
        "Lytic Agent": "dsmatdata:lyticagent_matfunc",
        "Skin Conditioning Agent - Humectant": "dsmatdata:skinconditioningagenthumectants_matfunc",
        "Flavoring Agent": "dsmatdata:flavoringagent_matfunc",
        "Excipient": "dsmatdata:excipient_matfunc",
        "Antifungal Agent": "dsmatdata:antifungalagent_matfunc"
    }}

- Only include material type IDs from the following mapping, default value is "dsmatdata:SourcedMaterial":
    {{
      "Experimental": "dsmatdata:ExperimentalMaterial",
      "Formulated": "dsmatdata:FormulatedMaterial",
      "Sourced": "dsmatdata:SourcedMaterial",
      "Unified": "dsmatdata:UnifiedMaterial"
    }}

- For sourced materials, set the tradeName to the product name on the datasheet, and use the following supplier name and ID:
    {{
      "supplierName": "{0}",
      "supplierId": "{1}"
    }}

- For groups and HasAlternative, keep the fields empty.
- For hasFormulationSet, only include fields if data is available on the datasheet.
- Only iclude Physical Form IDs from the following mapping:
    {{
      "Powder": "dsmatdata:powder_form",
      "Capsule": "dsmatdata:capsule_form",
      "Solid": "dsmatdata:solid_form",
      "Gel": "dsmatdata:gel_form",
      "Stick": "dsmatdata:stick_form",
      "Mousse": "dsmatdata:mousse_form",
      "Paste": "dsmatdata:paste_form",
      "Lotion": "dsmatdata:lotion_form",
      "Tablet": "dsmatdata:tablet_form",
      "Soft Solid": "dsmatdata:soft_solid_form",
      "Cream": "dsmatdata:cream_form",
      "Lozenge": "dsmatdata:lozenge_form",
      "Aerosol": "dsmatdata:aerosol_form",
      "Spray": "dsmatdata:spray_form",
      "Liquid": "dsmatdata:liquid_form",
      "Gas": "dsmatdata:gas_form",
      "Foam": "dsmatdata:foam_form"
    }}
  
- For UsageRestriction, set the ID to "{2}". For the message, provide a 1-line explanation if available; otherwise, use an empty string.
- For UsageStatus, set the ID to "{3}".
- For hasSecureCollection, set the ID to "dsmatdata:secure_collection_standard".
- Use true or false for boolean fields.
- Use null or [] when data is missing.
""".format(supplier_name, supplier_id, usage_restriction, usage_id)


In [29]:
print(system_prompt)


You are an expert materials data extraction model.
Your job is to analyze a material datasheet (provided as a PDF) and extract *all* structured material data
into a predefined JSON schema. Be precise and consistent.

Follow this JSON structure exactly, keeping the order of fields the same:
{
  "name": "str",
  "description": "str",
  "iupacName": "str",
  "casNumber": "str",
  "recipeNumber": "str",
  "synonym": ["str"],
  "hasAssignedIdentifier": [
    {
      "identifier": "str",
      "hasIdentifierType": {
        "id": "str"
      }
    }
  ],
  "hasConstituency": {
    "id": "str",
    "hasMixtureRatio": [
      {
        "ratioMaximum": "float",
        "ratioMinimum": "float",
        "ratiotarget": "float",
        "ratioMaximumInclusive": "bool",
        "ratioMinimumInclusive": "bool",
        "restOfMixture": "bool",
        "offLabel": "bool",
        "primaryConstituent": "bool",
        "hasActiveMaterialFunction": [
          {
            "id": "str"
          }
     

In [38]:
response = client.chat.completions.create(
    model="mistralai/Mistral-Small-3.2-24B-Instruct-2506", 
    messages=[
        {
            "role": "system", 
            "content": system_prompt
        },
        {
            "role": "user", 
            "content": "Extract material data for Anhydric Citric Acid"
        }
        ]
)

# for chunk in client.chat.completions.create(
#     model="mistralai/Mistral-Large-Instruct-2411", 
#     messages=[{"role": "user", "content": "Hello!"}], stream=True):
    
#     print(chunk.choices[0].delta)

In [39]:
print(response.choices[0].message.content)

```json
{
  "name": "Anhydric Citric Acid",
  "description": "Anhydrous citric acid is a white, crystalline powder that is highly soluble in water. It is a weak organic acid commonly used in food and beverage industries as a preservative, flavor enhancer, and acidulant.",
  "iupacName": "2-hydroxypropane-1,2,3-tricarboxylic acid",
  "casNumber": "77-92-9",
  "recipeNumber": "",
  "synonym": [
    "Citric Acid Anhydrous",
    "2-Hydroxy-1,2,3-propanetricarboxylic Acid",
    "1,2,3-Propanetricarboxylic Acid, 2-Hydroxy-",
    "Citric Acid Monohydrate (dehydrated)",
    "E330 (when used as a food additive)"
  ],
  "hasAssignedIdentifier": [],
  "hasConstituency": {
    "id": "",
    "hasMixtureRatio": []
  },
  "hasGrade": [
    {
      "id": "dsmatdata:laboratory_grade"
    }
  ],
  "hasMaterialFunction": [
    {
      "id": "dsmatdata:acidityregulator_matfunc"
    },
    {
      "id": "dsmatdata:preservative_matfunc"
    },
    {
      "id": "dsmatdata:flavoringagent_matfunc"
    }
  ],
